In [1]:
# Pyspark initialisation

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql import SQLContext

import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.master("local[2]").config("spark.driver.host","localhost").appName("test_harsha").getOrCreate()
sc = spark.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

spark

# Read parquet file in pyspark

In [3]:
import pyarrow
df=sqlContext.read.parquet("propellor-campaign-response-8-2020-02-11-09-24-17-2f664d1e-d83a-4433-b107-c57a2814b33d.parquet")

In [4]:
df

DataFrame[event_type: string, event_timestamp: bigint, arrival_timestamp: bigint, event_version: string, application: struct<app_id:string,sdk:string>, client: struct<client_id:string>, device: struct<platform:string>, session: string, attributes: struct<feedback:string>, awsaccountid: string, facets: struct<email_channel:struct<mail_event:struct<mail:struct<message_id:string,message_send_timestamp:bigint,from_address:string,destination:array<string>,headers_truncated:boolean,headers:array<struct<name:string,value:string>>,common_headers:struct<from:string,to:array<string>,subject:string>>,open:struct<ip_address:string,user_agent:string>>>>]

In [9]:
df.show()

+------------------+-------------------+-----------------+-------------+--------------------+-------------+------+-------+----------+------------+------+
|        event_type|    event_timestamp|arrival_timestamp|event_version|         application|       client|device|session|attributes|awsaccountid|facets|
+------------------+-------------------+-----------------+-------------+--------------------+-------------+------+-------+----------+------------+------+
|_test.event_stream|2020-02-11 14:54:17|             null|          3.1|[10428299d4064ac8...|[pinpointapi]|  [{}]|     {}|        []|        null|  null|
+------------------+-------------------+-----------------+-------------+--------------------+-------------+------+-------+----------+------------+------+



# Data cleaning

In [5]:
df=df.where(col("event_timestamp").isNotNull())

Type casting

In [6]:
df=df.withColumn('event_timestamp',col('event_timestamp').cast('string'))

# UDF 

In [7]:
get_timestamp = udf(lambda x:x[:10],StringType())

# String to timestamp UTC conversion

In [8]:
df=df.withColumn("event_timestamp", get_timestamp(df['event_timestamp']))
df = df.withColumn("event_timestamp", F.from_unixtime(col("event_timestamp"),'yyyy-MM-dd HH:mm:ss'))